In [1]:
import subprocess
subprocess.run(["wget", "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"])

--2024-12-28 16:44:31--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

     0K .......... .......... .......... .......... ..........  4% 3.12M 0s
    50K .......... .......... .......... .......... ..........  9% 4.24M 0s
   100K .......... .......... .......... .......... .......... 13% 16.8M 0s
   150K .......... .......... .......... .......... .......... 18% 16.9M 0s
   200K .......... .......... .......... .......... .......... 22% 6.25M 0s
   250K .......... .......... .......... .......... .......... 27% 31.5M 0s
   300K .......... .......... .......... .......... .......... 32% 28.9M 0s
   350K .......... 

CompletedProcess(args=['wget', 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'], returncode=0)

In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)


In [3]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]          # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [4]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)


/root/aiRoot/0-AI/AI/.venv/lib/python3.11/site-packages/torch/_subclasses/functional_tensor.py:295: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [5]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"for context: {context} the target is: {target}")

for context: tensor([18]) the target is: 47
for context: tensor([18, 47]) the target is: 56
for context: tensor([18, 47, 56]) the target is: 57
for context: tensor([18, 47, 56, 57]) the target is: 58
for context: tensor([18, 47, 56, 57, 58]) the target is: 1
for context: tensor([18, 47, 56, 57, 58,  1]) the target is: 15
for context: tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
for context: tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [25]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]  for i in ix])
    return x, y
    
xb, yb = get_batch('train')

tensor([ 76049, 234249, 934904, 560986])
